- **Nom**: David Morillo Massagué
- **NIU**: 1666540

### 1. Per a un sistema de xifratge RSA amb p = 97 i q = 31, quantes claus públiques podem fer servir?

In [4]:
from sympy import isprime
from sympy.ntheory import factorint
from sympy import gcd

p = 97
q = 31

n = p * q
phi_n = (p - 1) * (q - 1)
# Busquem els nombres primers que són coprims amb phi_n
def coprime_count(n):
    count = 0
    for i in range(2, n):
        if gcd(i, n) == 1:
            count += 1
    return count

coprime_count_phi_n = coprime_count(phi_n)
print(f"Nombre de claus públiques possibles: {coprime_count_phi_n}")

Nombre de claus públiques possibles: 767


### 2. Per un sistema de xifratge RSA amb p = 97 i q = 31, ens diuen que podem fer servir qualsevol de les següents claus públiques:
a) e = 24 b) e = 33 c) e = 45 d) e = 49.

Quines d’elles s´on millors i perquè?

In [6]:
# Comprovem quines claus públiques són vàlides
def is_valid_public_key(e, phi_n):
    return gcd(e, phi_n) == 1
valid_keys = [24, 33, 45, 49]
valid_keys = [e for e in valid_keys if is_valid_public_key(e, phi_n)]
print(f"Claus públiques vàlides: {valid_keys}")

Claus públiques vàlides: [49]


La única clau que seria vàlida per a utilitzar com a clau pública (e) és 49, ja que és la única que és coprima amb phi_n. Les altres claus públiques (24, 33 i 45) no són vàlides perquè no són coprimes amb phi_n (MCD(phi_n, e) != 1), i per tant no poden ser utilitzades com a claus públiques en el sistema RSA.

### 3. Els usuaris d’una xarxa que es comuniquen utilitzant el criptosistema RSA tenen les segúents claus públiques:

In [15]:
# (n, e)
pka = (979, 293)
pkb = (299, 217)
pkc = (407, 119)
pkd = (140030234401607803777917438251378213819219721524651118808434275263655395469528598704536324499566494098557043742549996566558639071657539815043385431186640932840798807785896997249571231108794361593482827908287632293375666323685398351507895710615198985712283304233326023055012524211849836100106559378736370665769, 65537)

pks = {
    'A': pka,
    'B': pkb,
    'C': pkc,
    'D': pkd
}

1. Calculeu les claus privades de A, B, C i D.

Calculats a mà:

A:
- n = 979 = 11x89
- φ(n) = 10x88 = 880
- e = 293
- d = e^-1 mod 880 = 877

B:
- n = 299 = 13x23
- φ(n) = 12x22 = 264
- e = 217
- d = e^-1 mod 264 = 73

C:
- n = 407 = 11x37
- φ(n) = 10x36 = 360
- e = 119
- d = e^-1 mod 360 = 239

D: Computacionalment inviable de calcular

2. Calculeu el xifratge del missatge m = 15 que B vol enviar a A.

In [25]:
sks = {
    'A': 877,  # Clau privada de A
    'B': 73,   # Clau privada de B
    'C': 239,  # Clau privada de C
}

m = 15

def encrypt_message(m, d, n):
    return pow(m, d, n)

priv_b = sks['B'] # 73
n = pks['B'][0] # 299
encrypted_message = encrypt_message(m, priv_b, n)
print(f"Missatge xifrat enviat de B a A: {encrypted_message}")

Missatge xifrat enviat de B a A: 80


3. Desxifreu el missatge rebut per A de B.

In [26]:
def decrypt_message(c, e, n):
    return pow(c, e, n)

pub_b = pks['B'][1]
n_b = pks['B'][0]

decrypted_message = decrypt_message(encrypted_message, pub_b, n_b)
print(f"Missatge desxifrat rebut per A de B: {decrypted_message}")

Missatge desxifrat rebut per A de B: 15


### 4. Suposem que els usuaris A i B porten a terme una distribuci´o de clau secreta utilitzant el protocol de Diffie-Hellman. Els valors que utilitzen són p = 7001, α = 101, a = 68 i b = 98.
Descriviu el protocol i calculeu la clau privada que comparteixen.

In [2]:
p = 7001
alpha = 101
a = 68
b = 98

def diffie_hellman(p, alpha, a, b):
    # A calcula la seva clau pública
    A_public = pow(alpha, a, p)
    print(f"Clau pública d'A: {A_public} (α^a mod p)")

    # B calcula la seva clau pública
    B_public = pow(alpha, b, p)
    print(f"Clau pública de B: {B_public} (α^b mod p)")

    print(f"A envia la seva clau pública a B: {A_public}")
    print(f"B envia la seva clau pública a A: {B_public}")

    # A calcula la clau compartida
    shared_key_A = pow(B_public, a, p)
    print(f"Clau compartida calculada per A: {shared_key_A} (α^b^a mod p)")

    # B calcula la clau compartida
    shared_key_B = pow(A_public, b, p)
    print(f"Clau compartida calculada per B: {shared_key_B} (α^a^b mod p)")

    return shared_key_A, shared_key_B

shared_key_A, shared_key_B = diffie_hellman(p, alpha, a, b)
print(f"\nClau compartida final: {shared_key_A} (A) = {shared_key_B} (B)")

Clau pública d'A: 176 (α^a mod p)
Clau pública de B: 2901 (α^b mod p)
A envia la seva clau pública a B: 176
B envia la seva clau pública a A: 2901
Clau compartida calculada per A: 2153 (α^b^a mod p)
Clau compartida calculada per B: 2153 (α^a^b mod p)

Clau compartida final: 2153 (A) = 2153 (B)


### 5. El protocol de Diffie-Hellman és vulnerable enfront d’atacs d’impersonació. Descriviu quins passos es modificarien i quins s’afegirien en l’intercanvi de claus de l’exercici anterior per tal que un atacant aconseguís compartir una clau diferent amb cada usuari malgrat els usuaris pensessin que comparteixen una sola clau entre ells dos.

In [4]:
def diffie_hellman_impersonation(p, alpha, a, b):
    # A calcula la seva clau pública
    A_public = pow(alpha, a, p)
    print(f"Clau pública d'A: {A_public} (α^a mod p)")

    # L'atacant intercepta la clau d'A i n'envia una falsa a B
    print(f"L'atacant intercepta la clau pública d'A: {A_public}")
    fake_A_public = (A_public + 1) % p
    print(f"Atacant envia una clau falsa a B: {fake_A_public}")

    # B calcula la seva clau pública
    B_public = pow(alpha, b, p)
    print(f"Clau pública de B: {B_public} (α^b mod p)")

    # B calcula la clau compartida amb la clau falsa d'A
    shared_key_B = pow(fake_A_public, b, p)
    print(f"Clau compartida calculada per B amb la clau falsa d'A: {shared_key_B} ((clau falsa)^b mod p)")

    # L'atacant intercepta la clau de B i n'envia una falsa a A
    print(f"L'atacant intercepta la clau pública de B: {B_public}")
    fake_B_public = (B_public + 1) % p
    print(f"Atacant envia una clau falsa a A: {fake_B_public}")

    # A calcula la clau compartida amb la clau falsa de B
    shared_key_A = pow(fake_B_public, a, p)
    print(f"Clau compartida calculada per A amb la clau falsa de B: {shared_key_A} ((clau falsa)^a mod p)")

    return shared_key_A, shared_key_B

shared_key_A_imp, shared_key_B_imp = diffie_hellman_impersonation(p, alpha, a, b)
print(f"\nClau compartida final amb atac d'impersonació:")
print(f"A rep: {shared_key_A_imp}")
print(f"B rep: {shared_key_B_imp}")

Clau pública d'A: 176 (α^a mod p)
L'atacant intercepta la clau pública d'A: 176
Atacant envia una clau falsa a B: 177
Clau pública de B: 2901 (α^b mod p)
Clau compartida calculada per B amb la clau falsa d'A: 1149 ((clau falsa)^b mod p)
L'atacant intercepta la clau pública de B: 2901
Atacant envia una clau falsa a A: 2902
Clau compartida calculada per A amb la clau falsa de B: 2153 ((clau falsa)^a mod p)

Clau compartida final amb atac d'impersonació:
A rep: 2153
B rep: 1149


A i B creuen que comparteixen una sola clau, però en realitat l'atacant ha interceptat les claus públiques i ha enviat claus falses a cada usuari. L'atacant pot rebre missatges de A i B, i pot enviar missatges falsos a cada usuari sense que ells ho sàpiguen. Això permet a l'atacant controlar la comunicació entre A i B, fent que creguin que estan parlant directament entre ells quan en realitat estan parlant amb l'atacant.